# Laboratorio 1

## Integrantes

- Abby Donis
- Cindy Gualim
- Josué Say - 22801

## Repositorio

- [Enlace a GitHub](https://github.com/JosueSay/labs-ms/tree/main/Labs/lab1)

## Preparación de entorno

Usar si se está trabajando en lab:

In [2]:
using Pkg
Pkg.activate(".")

Pkg.add("JuMP")
# Instalar otros paquetes necesarios...

Pkg.activate()
Pkg.status()

  Activating new project at `d:\repositorios\UVG\2025\labs-ms\Labs\lab1`
    Updating registry at `C:\Users\josue\.julia\registries\General.toml`
   Resolving package versions...
   Installed DiffRules ──────────── v1.15.1
   Installed SpecialFunctions ───── v2.5.1
   Installed Bzip2_jll ──────────── v1.0.9+0
   Installed DiffResults ────────── v1.1.0
   Installed JSON3 ──────────────── v1.14.3
   Installed IrrationalConstants ── v0.2.4
   Installed CodecBzip2 ─────────── v0.8.5
   Installed MutableArithmetics ─── v1.6.4
   Installed StaticArraysCore ───── v1.4.3
   Installed NaNMath ────────────── v1.1.3
   Installed BenchmarkTools ─────── v1.6.0
   Installed ForwardDiff ────────── v1.0.1
   Installed CommonSubexpressions ─ v0.3.1
   Installed TranscodingStreams ─── v0.11.3
   Installed LogExpFunctions ────── v0.3.29
   Installed MacroTools ─────────── v0.5.16
   Installed OpenSpecFun_jll ────── v0.5.6+0
   Installed CodecZlib ──────────── v0.7.8
   Installed DocStringExtensions ── v0

Status

  10909.5 ms  ✓ JuMP
  23 dependencies successfully precompiled in 62 seconds. 28 already precompiled.
  Activating project at `C:\Users\josue\.julia\environments\v1.11`


 `C:\Users\josue\.julia\environments\v1.11\Project.toml`
  [7073ff75] IJulia v1.29.0


Usar si ya se ha terminado el lab:

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.status()